### Introduction/Business Problem


I want to open a fast food restaurant in Berlin, and I am looking to find the best city for my new venue. 
For that I must do some research on different locations as well as competitors in those locations:
My location must be near to be near to offices, universities and schools to be sure having enough customers
but not too close to other competitors.

I will be using Foursquare to gather information about different competitors in different locations in Germany
and then predict some statistics for a new venue depending on restaurant category and location.

### Data

For my use case I will need the following data to determine which locality is the most appropriate for my new venue:
-	The coordinates (latitude, longitude) of Berlin
-	The Fast Food venues in Berlin
-	The offices venues in Berlin
-	The high schools, universities venues in Berlin
- The shopping malls in Berlin

### Methodology

I plotted all the venues in Berlin and visually examined where there are enough offices, universities and schools and at the same time not too much competitors.

### Praktisches Teil

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'K4LZ5SJRJNTMSMDZYCWOBJYA34ELZWRCT2QPKKQDEXS2D5LS' # your Foursquare ID
CLIENT_SECRET = 'ZSPJQCFMEVFFCZZ3QMCR2NNQV0OC5WDXC0TE21EHO3KJ1215' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

### Berlin, Germany latitude longitude

In [4]:
latitude = 52.52437
longitude = 13.41053

In [18]:
search_query = 'fastfood'

In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=K4LZ5SJRJNTMSMDZYCWOBJYA34ELZWRCT2QPKKQDEXS2D5LS&client_secret=ZSPJQCFMEVFFCZZ3QMCR2NNQV0OC5WDXC0TE21EHO3KJ1215&ll=52.52437,13.41053&v=20180604&query=fastfood&limit=30'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8b1b72db1d81002cb83585'},
 'response': {'venues': []}}

In [16]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

""


In [17]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

KeyError: "None of [Index(['name', 'categories', 'id'], dtype='object')] are in the [columns]"

In [10]:
dataframe_filtered.name

0               Italian Fast Food
1                Vietanh Fastfood
2                       Fast Food
3                   Asia Fastfood
4         Asia-Fast-Food Tienminh
5      Viet Asia - Asia Fast Food
6        BIBI Fastfood Restaurant
7                   Asia Fastfood
8              Viet Anh Fast Food
9                 Enjoy Fast Food
10                   Auf die Hand
11    M Burger -American Fastfood
12                       Halle 18
13     Kabanos - Polska Fast Food
14                 Asia Fast Food
15              Lalanga Fast Food
16                Namky Fast food
17               Asia - Fast Food
18         SNAP Natural Fast Food
19               B-Town Fast Food
20               Saigon Fast Food
21                 Asia Fast Food
22                 Asia Fast Food
Name: name, dtype: object

In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add the Fastfood restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map